In [1]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
df=pd.read_csv("fake_or_real_news.csv")

In [3]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
Unnamed: 0    6335 non-null int64
title         6335 non-null object
text          6335 non-null object
label         6335 non-null object
dtypes: int64(1), object(3)
memory usage: 198.0+ KB


In [5]:
y=df.label

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_text=train_test_split(df['text'],y,test_size=0.33,random_state=53)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer=CountVectorizer(stop_words='english')

In [8]:
count_train=count_vectorizer.fit_transform(X_train)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_df=0.7)

In [11]:
tf_idf_train=tfidf_vectorizer.fit_transform(X_train)

In [14]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(tfidf_df,y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
tf_idf_test=tfidf_vectorizer.transform(X_test)
#model.score(tf_idf_test,y_text)

In [16]:
from sklearn.ensemble import AdaBoostClassifier
clf=AdaBoostClassifier()
clf.fit(tf_idf_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [17]:
clf.score(tf_idf_test,y_text)

0.86657101865136299

In [18]:
count_test=count_vectorizer.transform(X_test)

In [19]:
model.fit(count_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
model.score(count_test,y_text)

0.89335246293639403

In [21]:
clf.fit(count_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [22]:
clf.score(count_test,y_text)

0.85078909612625542

In [23]:
clf=AdaBoostClassifier(MultinomialNB())
clf.fit(count_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
          learning_rate=1.0, n_estimators=50, random_state=None)

In [24]:
clf.score(count_test,y_text)

0.71353419416547104

In [25]:
combined_train=np.concatenate([count_train.A,tf_idf_train.A],axis=1) #combining both count and tfidf vectors

In [26]:
model.fit(combined_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
combined_test=np.concatenate([count_test.A,tf_idf_test.A],axis=1)

In [28]:
model.score(combined_test,y_text)

0.8947871831659493

In [29]:
y_text.value_counts()

REAL    1083
FAKE    1008
Name: label, dtype: int64

In [30]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_dict = {'kernel' :['linear','rbf'],'C' : [0.01,1,10,100]} 
grid=GridSearchCV(SVC(),param_dict,cv=5)
grid.fit(count_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ['linear'], 'C': [1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [31]:
grid.score(count_test,y_text)

0.87039693926351025

In [32]:
grid.fit(tf_idf_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ['linear'], 'C': [1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [33]:
grid.predict(tf_idf_test)

array(['REAL', 'REAL', 'REAL', ..., 'REAL', 'FAKE', 'REAL'], dtype=object)

In [34]:
grid.best_params_

{'C': 1, 'kernel': 'linear'}

In [35]:
grid.score(tf_idf_test,y_text)

0.93448110951697749

In [36]:
from sklearn.ensemble import RandomForestClassifier
param_grid={'n_estimators' : [50,75,100], 'min_samples_leaf' : [30,50,70]}
model=RandomForestClassifier()
grid=GridSearchCV(model,param_grid,cv=5)
grid.fit(count_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [50, 75, 100], 'min_samples_leaf': [30, 50, 70]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [37]:
grid.best_params_

{'min_samples_leaf': 30, 'n_estimators': 50}

In [38]:
grid.score(count_test,y_text)

0.81252989000478237

In [40]:
from sklearn.ensemble import RandomForestClassifier
param_grid={'n_estimators' : [50,75,100], 'min_samples_leaf' : [30,50,70]}
model=RandomForestClassifier()
grid=GridSearchCV(model,param_grid,cv=5)
grid.fit(tf_idf_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [50, 75, 100], 'min_samples_leaf': [30, 50, 70]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [41]:
grid.score(tf_idf_test,y_text)

0.8316594930655189

In [48]:
### SVC is the best model, I will try to improve it using scaling
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler,Normalizer,StandardScaler
pipe=Pipeline([('sc',Normalizer()),('svc',LinearSVC())])
param_dict = {'svc__C' : [0.01,1,10,100]}
grid=GridSearchCV(pipe,param_dict,cv=5)
grid.fit(tf_idf_train.A,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('sc', Normalizer(copy=True, norm='l2')), ('svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'svc__C': [0.01, 1, 10, 100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [45]:
grid.best_score_

0.91870876531573986

In [47]:
grid.score(tf_idf_test.A,y_text)

0.91678622668579624

In [49]:
grid.best_params_

{'svc__C': 1}

### Scaling didn't improve the accuracy

In [52]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
param_dict = {'C' : [0.01,1,10,100]} 
grid=GridSearchCV(LinearSVC(),param_dict,cv=5)
grid.fit(combined_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.01, 1, 10, 100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [54]:
grid.score(combined_test,y_text) #combining both tfidf and count didn't improve the accuracy

0.91869918699186992

In [55]:
#the best model is linearSVC on the tfidf vectors with 94% accuracy
svc=LinearSVC()
svc.fit(tf_idf_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [56]:
svc.score(tf_idf_test,y_text)

0.93639406982305118

In [58]:
svc.score(tf_idf_train,y_train)

0.99976437323279921

In [60]:
#Since the svc model is over fitting, I am using Bagging to improve the model
from sklearn.ensemble import BaggingClassifier
param_grid={'n_estimators' : [50,100,200]}
clf=BaggingClassifier(LinearSVC())
grid=GridSearchCV(clf,param_grid,cv=5)
grid.fit(tf_idf_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=BaggingClassifier(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [50, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [61]:
grid.score(tf_idf_test,y_text)

0.93639406982305118

### Bagging didn't improve the model, I should ideally increase the number of estimators to reduce the overfit, however it is taking a lot of time to run